In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder

from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC

from sklearn.model_selection import train_test_split,cross_val_score

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

from sklearn.ensemble import IsolationForest

from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, classification_report, roc_curve, plot_roc_curve, auc, precision_recall_curve, plot_precision_recall_curve, average_precision_score

In [ ]:
df = pd.read_csv('healthcare-dataset-stroke-data.csv')

In [ ]:
#Remove the ID column
df.info()
df.drop('id', axis=1, inplace=True)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [ ]:
DT_bmi_pipe = Pipeline( steps=[ 
                               ('scale',StandardScaler()),
                               ('lr',DecisionTreeRegressor(random_state=0))
                              ])
X = df[['age','gender','bmi']].copy()
X.gender = X.gender.replace({'Male':0,'Female':1,'Other':-1}).astype(np.uint8)

Missing = X[X.bmi.isna()]
X = X[~X.bmi.isna()]
Y = X.pop('bmi')
DT_bmi_pipe.fit(X,Y)
predicted_bmi = pd.Series(DT_bmi_pipe.predict(Missing[['age','gender']]),index=Missing.index)
df.loc[Missing.index,'bmi'] = predicted_bmi

In [ ]:
##Label Encode Binary
residence_type = df['Residence_type'].unique()
ever_married = df['ever_married'].unique()
le = LabelEncoder()

le.fit(residence_type)
df['Residence_type'] = le.transform(df['Residence_type'])

le.fit(ever_married)
df['ever_married'] = le.transform(df['ever_married'])

In [ ]:
df
#Label Encode -> One Hot Encode -> PCA -> Splitting -> Outlier Detection -> SMOTE X_train and y_train -> Bayes Training

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,1,Private,1,228.69,36.600000,formerly smoked,1
1,Female,61.0,0,0,1,Self-employed,0,202.21,29.879487,never smoked,1
2,Male,80.0,0,1,1,Private,0,105.92,32.500000,never smoked,1
3,Female,49.0,0,0,1,Private,1,171.23,34.400000,smokes,1
4,Female,79.0,1,0,1,Self-employed,0,174.12,24.000000,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
5105,Female,80.0,1,0,1,Private,1,83.75,28.476923,never smoked,0
5106,Female,81.0,0,0,1,Self-employed,1,125.20,40.000000,never smoked,0
5107,Female,35.0,0,0,1,Self-employed,0,82.99,30.600000,never smoked,0
5108,Male,51.0,0,0,1,Private,0,166.29,25.600000,formerly smoked,0


In [ ]:
X = df[['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 
       'avg_glucose_level', 'bmi', 'smoking_status']]
y = df['stroke']

In [ ]:
#One Hot Encode everything
encoder = ColumnTransformer(transformers=[('onehot', OneHotEncoder(), ['gender', 'work_type', 'smoking_status'])], remainder='passthrough')
X = np.array(encoder.fit_transform(X))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
#Standardize
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
#Isolation Forest
iso = IsolationForest(contamination=0.1)
yhat = iso.fit_predict(X_train)

In [ ]:
mask = yhat != -1
X_train, y_train = X_train[mask, :], y_train[mask]

In [ ]:
#SMOTE
X_smote, y_smote = SMOTE().fit_resample(X_train, y_train)

# ANN

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from keras.models import Sequential  
from keras.layers import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import regularizers

In [ ]:
def ann_classifier(optimizer = 'adam'):
    model = tf.keras.models.Sequential()
    model.add(layers.Dense(8, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01), activation=tf.nn.relu))
    model.add(layers.Dense(8, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01), activation=tf.nn.relu))
    tf.keras.layers.Dropout(0.6)
    model.add(layers.Dense(1, activation=tf.nn.sigmoid))
    model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
    return model

In [ ]:
ann = KerasClassifier(build_fn = ann_classifier, batch_size = 32, epochs = 50)

# Before tuning

In [ ]:
prior_model = tf.keras.models.Sequential()
prior_model.add(layers.Dense(8, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01), activation=tf.nn.relu))
prior_model.add(layers.Dense(8, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01), activation=tf.nn.relu))
tf.keras.layers.Dropout(0.4)
prior_model.add(layers.Dense(1, activation=tf.nn.sigmoid))
prior_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
epochs = 100
history = prior_model.fit(X_smote, y_smote, epochs=epochs)

Epoch 1/100
221/221 [==============================] - 1s 468us/step - loss: 0.8192 - accuracy: 0.5768
Epoch 2/100
221/221 [==============================] - 0s 459us/step - loss: 0.6241 - accuracy: 0.7419
Epoch 3/100
221/221 [==============================] - 0s 464us/step - loss: 0.5336 - accuracy: 0.7929
Epoch 4/100
221/221 [==============================] - 0s 464us/step - loss: 0.4964 - accuracy: 0.8038
Epoch 5/100
221/221 [==============================] - 0s 468us/step - loss: 0.4772 - accuracy: 0.8092
Epoch 6/100
221/221 [==============================] - 0s 459us/step - loss: 0.4670 - accuracy: 0.8113
Epoch 7/100
221/221 [==============================] - 0s 468us/step - loss: 0.4602 - accuracy: 0.8119
Epoch 8/100
221/221 [==============================] - 0s 468us/step - loss: 0.4545 - accuracy: 0.8143
Epoch 9/100
221/221 [==============================] - 0s 464us/step - loss: 0.4512 - accuracy: 0.8171
Epoch 10/100
221/221 [==============================] - 0s 446us/step - l

221/221 [==============================] - 0s 486us/step - loss: 0.3720 - accuracy: 0.8576
Epoch 80/100
221/221 [==============================] - 0s 485us/step - loss: 0.3721 - accuracy: 0.8567
Epoch 81/100
221/221 [==============================] - 0s 477us/step - loss: 0.3696 - accuracy: 0.8635
Epoch 82/100
221/221 [==============================] - 0s 482us/step - loss: 0.3689 - accuracy: 0.8586
Epoch 83/100
221/221 [==============================] - 0s 473us/step - loss: 0.3698 - accuracy: 0.8573
Epoch 84/100
221/221 [==============================] - 0s 468us/step - loss: 0.3676 - accuracy: 0.8629
Epoch 85/100
221/221 [==============================] - 0s 468us/step - loss: 0.3677 - accuracy: 0.8588
Epoch 86/100
221/221 [==============================] - 0s 459us/step - loss: 0.3674 - accuracy: 0.8603
Epoch 87/100
221/221 [==============================] - 0s 459us/step - loss: 0.3669 - accuracy: 0.8625
Epoch 88/100
221/221 [==============================] - 0s 468us/step - loss:

In [ ]:
y_pred=(prior_model.predict(X_test) > 0.5).astype("int32")

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, y_pred)
print('F1 score: %f' % f1)
roc = roc_auc_score(y_test, y_pred)
print('ROC score: %f' % roc)
print('----------------------------')
print(classification_report(y_test, y_pred))

Accuracy: 0.748532
Precision: 0.114068
Recall: 0.555556
F1 score: 0.189274
ROC score: 0.657427
----------------------------
              precision    recall  f1-score   support

           0       0.97      0.76      0.85       968
           1       0.11      0.56      0.19        54

    accuracy                           0.75      1022
   macro avg       0.54      0.66      0.52      1022
weighted avg       0.92      0.75      0.82      1022



# Tune ANN

In [ ]:
# parameters = {'batch_size': [25, 32],
#              'epochs': [50, 100, 150],
#              'optimizer': ['adam', 'rmsprop']}

# grid_search = GridSearchCV(estimator = ann, param_grid = parameters, scoring = 'accuracy', cv = 10, n_jobs = -1)

# grid_search.fit(X_smote, y_smote)

In [ ]:
# best_accuracy = grid_search.best_score_
# best_parameters = grid_search.best_params_
# print("Best ROC AUC: {:.2f} %".format(best_accuracy*100))
# print("Best Parameters:", best_parameters)

# Running ANN after Tuning

In [ ]:
model = tf.keras.models.Sequential()
model.add(layers.Dense(8, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01), activation=tf.nn.relu))
model.add(layers.Dense(8, kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01), activation=tf.nn.relu))
tf.keras.layers.Dropout(0.4)
model.add(layers.Dense(1, activation=tf.nn.sigmoid))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
epochs = 150
batch_size = 25
history = model.fit(X_smote, y_smote, batch_size=25, epochs=epochs)

Epoch 1/150
283/283 [==============================] - 3s 1ms/step - loss: 0.8046 - accuracy: 0.6478
Epoch 2/150
283/283 [==============================] - 0s 1ms/step - loss: 0.5787 - accuracy: 0.7733
Epoch 3/150
283/283 [==============================] - 0s 1ms/step - loss: 0.5007 - accuracy: 0.8092
Epoch 4/150
283/283 [==============================] - 0s 1ms/step - loss: 0.4734 - accuracy: 0.8195
Epoch 5/150
283/283 [==============================] - 0s 1ms/step - loss: 0.4570 - accuracy: 0.8255
Epoch 6/150
283/283 [==============================] - 0s 1ms/step - loss: 0.4452 - accuracy: 0.8313
Epoch 7/150
283/283 [==============================] - 0s 1ms/step - loss: 0.4361 - accuracy: 0.8351
Epoch 8/150
283/283 [==============================] - 0s 1ms/step - loss: 0.4297 - accuracy: 0.8403
Epoch 9/150
283/283 [==============================] - 0s 1ms/step - loss: 0.4246 - accuracy: 0.8421
Epoch 10/150
283/283 [==============================] - 0s 1ms/step - loss: 0.4201 - accura

283/283 [==============================] - 0s 1ms/step - loss: 0.3309 - accuracy: 0.8784
Epoch 82/150
283/283 [==============================] - 0s 1ms/step - loss: 0.3301 - accuracy: 0.8808
Epoch 83/150
283/283 [==============================] - 0s 1ms/step - loss: 0.3293 - accuracy: 0.8799
Epoch 84/150
283/283 [==============================] - 0s 1ms/step - loss: 0.3304 - accuracy: 0.8799
Epoch 85/150
283/283 [==============================] - 0s 1ms/step - loss: 0.3290 - accuracy: 0.8809
Epoch 86/150
283/283 [==============================] - 0s 1ms/step - loss: 0.3281 - accuracy: 0.8792
Epoch 87/150
283/283 [==============================] - 0s 1ms/step - loss: 0.3275 - accuracy: 0.8826
Epoch 88/150
283/283 [==============================] - 0s 1ms/step - loss: 0.3276 - accuracy: 0.8796
Epoch 89/150
283/283 [==============================] - 0s 1ms/step - loss: 0.3271 - accuracy: 0.8795
Epoch 90/150
283/283 [==============================] - 0s 1ms/step - loss: 0.3276 - accuracy: 

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)

32/32 [==============================] - 1s 1ms/step - loss: 0.5716 - accuracy: 0.7730


In [ ]:
y_pred=(model.predict(X_test) > 0.5).astype("int32")

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, y_pred)
print('F1 score: %f' % f1)
roc = roc_auc_score(y_test, y_pred)
print('ROC score: %f' % roc)
print('----------------------------')
print(classification_report(y_test, y_pred))

Accuracy: 0.772994
Precision: 0.113043
Recall: 0.481481
F1 score: 0.183099
ROC score: 0.635369
----------------------------
              precision    recall  f1-score   support

           0       0.96      0.79      0.87       968
           1       0.11      0.48      0.18        54

    accuracy                           0.77      1022
   macro avg       0.54      0.64      0.53      1022
weighted avg       0.92      0.77      0.83      1022

